In [11]:
import os
import json
import spacy
import numpy as np
import random
import torch

In [12]:
file = "../mnt/data/objaverse-processed/merged_for_training_final/Objaverse/000-152/e40a05dbe3704872bcc9dd49593c41d5.npy"
data = np.load(file, allow_pickle=True).item()

In [13]:
def get_cat_idx():
    objaverse_dict = torch.load("../src/eval_data/objaverse_dict.pt")
    ov_category2idx = objaverse_dict["category2idx"]
    ov_idx2category = objaverse_dict["idx2category"]

    return ov_category2idx, ov_idx2category

In [14]:
test_file = "/projectnb/ivc-ml/harshk/3d_perception/OpenShape3D/meta_data/split/lvis.json"
ov_category2idx, ov_idx2category = get_cat_idx()

In [46]:
# For finetuning 10 categories
# cat_list = [464, 409, 861, 372, 510, 106, 548, 1019, 1020, 562]

# cat_list = [328, 653] # desk - monitor_(computer_equipment) computer_monitor
# cat_list = [29, 515] # armor - helmet
# cat_list = [221, 1020] # chinaware - teapot
cat_list = [20, 1138] # antenna - windmill

In [47]:
for cat in cat_list:
    print(ov_idx2category[cat])

antenna
windmill


In [48]:
data_list = json.load(open(test_file, "r"))


In [49]:
filter_data = {}
for i in cat_list:
    filter_data[i] = []

for obj in data_list:
    cat_id = ov_category2idx[obj["category"]]
    if cat_id in cat_list:
        filter_data[cat_id].append(obj)
        # filter_data.append(obj)
        # print(obj["category_id"], obj["category_name"])

In [50]:
dir = "../finetune/antenna_windmill/"
if not os.path.exists(dir):
    os.makedirs(dir)
file_name = os.path.join(dir, "lvis_filtered.json")

json.dump(filter_data, open(file_name, "w"), indent=4)

In [51]:
train_split = 0.8
train_data = {}
test_data = {}

for key, val in filter_data.items():
    # print(key, len(val))
    data_size = len(val)
    random.shuffle(val)
    train_size = int(train_split * data_size)
    train_data[key] = val[:train_size]
    test_data[key] = val[train_size:]

json.dump(train_data, open(os.path.join(dir, "lvis_filter_train.json"), "w"), indent=4)
json.dump(test_data, open(os.path.join(dir, "lvis_filter_test.json"), "w"), indent=4)
# json.dump(train_data, open("finetune/lvis_filter_train.json", "w"), indent=4)
# json.dump(test_data, open("finetune/lvis_filter_test.json", "w"), indent=4)

In [ ]:
nlp = spacy.load('en_core_web_sm') 
all_text = json.load(open('all_text.json'))

In [ ]:
for text in random.choices(all_text, k=3):
    doc = nlp(text)
    print("text: ", text)
    noun = None
    adj = None
    for token in doc:
        if token.pos_ == "NOUN":
            noun = token.text
        elif token.pos_ == "ADJ":
            adj = token.text
        print(f"{token.text}: {token.pos_}")
    # print("noun: ", noun, "\tadj: ", adj)
    print("\n")
        # print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
        #         token.shape_, token.is_alpha, token.is_stop)
    # all_text[i] = cats